IMPORTANT INSTRUCTIONS: To use the EV optimizer widget at the bottom of the page, select the code cell above it and press SHIFT-ENTER. A number of controllable widgets should appear, along with a button that says 'calculate'. Hit calculate to run the optimization function

The text below explains what the widget is doing

***************************************

When we give EVs to a defensive pokemon, we want to maximize the amount of damage it can take before fainting

One way to optimize EVs is to do calculations with specific attacks (e.g. jolly excadrill max-steelspike), and tailor EVs to survive them. However, this is difficult to do if you are not sure what kind of attacks to expect

Without specific attacks to base EVs on, we use heuristics. Most people have the general sense that adding to HP is the best, and then the optimal spread across Def/SpD depends on what kind of hits the pokemon is expecting to take. Can we quantify these instincts in a rigorous way?

Simplifying the damage formula a bit, we can assume that if a pokemon takes X damage of which ratio P is physical and (1-P) is special, it will take an amount of damage equal to $\frac{X * P}{Def} + \frac{X*(1-P)}{SpD}$. If this is at or above the pokemon's HP, the pokemon will faint. We want to maximize the value of X needed to faint the pokemon,  so we can define our problem as $max(X)$ where $\frac{X * P}{Def} + \frac{X*(1-P)}{SpD} = HP$ and $HP$, $Def$, and $SpD$ are constrained by the limits of EV and IV training

Unfortunately, the constraint is non-convex and thus the problem is difficult to optimize explicitly. Fortunately, the solution space is rather small, and a computer can quickly optimize this function by checking every sensible input. 

If we solve for X as a function of $HP$/$Def$/$SpD$, we can query the function for different EV and nature values and see which combination yields the highest X

\begin{equation}
\frac{X*P}{Def} + \frac{X*(1-P)}{SpD} = HP \\[10pt]
\frac{X*P*SpD + X*(1-P)*Def}{Def*SpD} = HP \\[10pt]
X*P*SpD + X*(1-P)*Def = HP*Def*SpD \\[10pt]
X*(P*SpD + (1-P)*Def) = HP*Def*SpD \\[10pt]
X = \frac{HP*Def*SpD}{P*SpD + (1-P)*Def} \\[10pt]
\end{equation}

The function below iterates through sensible EV and nature choices, and spits out the set that maximizes X

In [23]:
import ipywidgets as widgets
from IPython.display import display

individual_ev_max = 252

def optimize_evs(hp, defense, special_defense, ev_max, individual_ev_max, physical_fraction, nature):
    
    best_damage_taken = 0

    for hp_investment in range(0,int((min(individual_ev_max,ev_max)+4)/8 + 1)):
        hp_evs = max(hp_investment*8-4,0)

        for defense_investment in range(0, int((min(individual_ev_max,ev_max-hp_evs)+4)/8 + 1)):
            
            defense_evs = max(defense_investment*8-4,0)
            
            special_defense_evs = max(min(individual_ev_max,ev_max-hp_evs-defense_evs),0)
            special_defense_investment = int((special_defense_evs+4)/8)

            hp_total = hp + hp_investment
            
            for defense_nature in [True, False] if nature else [False]:
                
                defense_total = defense + defense_investment
                special_defense_total = special_defense + special_defense_investment 
                
                if defense_nature:
                    defense_total = int(defense_total*1.1)
                elif nature:
                    special_defense_total = int(special_defense_total*1.1)
                    
                damage_taken = hp_total*defense_total*special_defense_total/ \
                                ((1-physical_fraction)*defense_total +(physical_fraction)*special_defense_total)

                if damage_taken > best_damage_taken:
                    best_damage_taken = damage_taken

                    best_defense_evs = defense_evs
                    best_special_defense_evs = special_defense_evs
                    best_hp_evs = hp_evs
                    best_nature = 'D' if defense_nature else 'SpD' if nature else None
            
    return best_hp_evs, best_defense_evs, best_special_defense_evs, best_nature

style = {'description_width': 'initial'}

hp = widgets.BoundedIntText(min = 0,
                            max = individual_ev_max,
                            step = 4,
                            value=160,
                            description='HP:',
                            disabled=False,
                            style = style
                        )
defense = widgets.BoundedIntText(min = 0,
                            max = individual_ev_max,
                            step = 4,
                            value=110,
                            description='Defense:',
                            disabled=False,
                            style = style

                        )
special_defense = widgets.BoundedIntText(
                            min = 0,
                            max = individual_ev_max,
                            step = 4,
                            value=115,
                            description='Special Defense:',
                            disabled=False,
                            style = style

                        )

ev_max = widgets.BoundedIntText(
                            value=508,
                            min = 0,
                            max = 508,
                            step = 4,
                            description='EVs available:',
                            disabled=False,
                            style = style

                        )

physical_fraction = widgets.FloatSlider(
                            min=0,
                            max=1,
                            step = 0.01,
                            readout_format='.2f',
                            value = 0.6,
                            description='Physical Fraction:',
                            disabled=False,
                            style = style
                        )

nature = widgets.Checkbox(
                            value= True,
                            description='Use Defensive Nature',
                            disabled=False,
                            style = style

                        )

button = widgets.Button(description = 'Calculate',
                       style = style)

hp_output = widgets.Text(description = 'HP EVs:'
                        ,disabled = True
                        ,style = style)

defense_output = widgets.Text(description = 'Defense EVs:'
                        ,disabled = True
                        ,style = style)

special_defense_output = widgets.Text(description = 'Special defense EVs:'
                        ,disabled = True
                        ,style = style)

def on_button_clicked(b):
    
    hp_val, def_val, spd_val, n = optimize_evs(hp.value
                               ,defense.value
                               ,special_defense.value
                               ,ev_max.value
                               ,individual_ev_max
                               ,physical_fraction.value
                               ,nature.value
                )
    hp_output.value = str(hp_val)
    defense_output.value = str(def_val) + '+' if n == 'D' else str(def_val)
    special_defense_output.value = str(spd_val) + '+' if n == 'SpD' else str(spd_val)

button.on_click(on_button_clicked)

display(hp)
display(defense)
display(special_defense)
display(ev_max)
display(nature)
display(physical_fraction)
display(button)
display(hp_output)
display(defense_output)
display(special_defense_output)

BoundedIntText(value=160, description='HP:', max=252, step=4, style=DescriptionStyle(description_width='initia…

BoundedIntText(value=110, description='Defense:', max=252, step=4, style=DescriptionStyle(description_width='i…

BoundedIntText(value=115, description='Special Defense:', max=252, step=4, style=DescriptionStyle(description_…

BoundedIntText(value=508, description='EVs available:', max=508, step=4, style=DescriptionStyle(description_wi…

Checkbox(value=True, description='Use Defensive Nature', style=DescriptionStyle(description_width='initial'))

FloatSlider(value=0.6, description='Physical Fraction:', max=1.0, step=0.01, style=SliderStyle(description_wid…

Button(description='Calculate', style=ButtonStyle())

Text(value='', description='HP EVs:', disabled=True, style=DescriptionStyle(description_width='initial'))

Text(value='', description='Defense EVs:', disabled=True, style=DescriptionStyle(description_width='initial'))

Text(value='', description='Special defense EVs:', disabled=True, style=DescriptionStyle(description_width='in…

The default values are for my porygon2. With perfect IV's and no EV's, my porygon2 has 160 HP, 110 Defense, and 115 Special defense. I expect it to take 60% of opposing offense from physical attacks, because porygon2 is weak to fighting and fighting attacks are mostly physical. 

Hit the 'calculate' button with the default parameters to get a spread for my porygon2. the spread should be 252 HP, 236+ Def (the + indicates a physically defensive nature such as bold), and 20 SpD. This makes intuitive sense because we expect more physical attacks and porygon2's defense starts lower than its special defense. 

Now if I feel like it, I can look up specific calculations and make adjustments. For example, I am curious about porygon-Z. I see that a modest life orb porygon-Z has a 25% chance of 0HKOing this porygon. Porygon-Z is 8% of the metagame, and roughly half are modest, so this will be relevant 4% of the time and result in an 0HKO 1% of the time if PZ always attacks P2. Adding 16 SpD EVs would reduce to chance of an 0HKO to 12.5%, so I might change the spread manually to (252,220,36,' + Defense Nature') if I am concerned about this